In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
device = "cuda" if torch.cuda.is_available() else "cpu"
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 1000

In [2]:
with open("x.txt", "r", encoding="UTF-8") as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

In [3]:
str_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_str = { i:ch for i, ch in enumerate(chars) }
encode = lambda str: [str_to_int[ch] for ch in str]
decode = lambda ints: "".join([int_to_str[i] for i in ints])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [5]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            return logits, None
        
        # Reshape tensor to calculate loss
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]

            # Generate probabilities using logits and select a character based on that
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)

        return index

In [6]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# New line character to start
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


)—3WJ6â 1y7ãK8J?4ïoVxHV:D]
6dgb”a”5]m:8bFág33”ÉUuTIS733,Xn88c;’gfikçSQiçHXÉÉl?n]mPâyFZ”F5PfB2‘N/Xj!1…ioñJMCOöñz7hWF(6sZñsö
xtá8:76Vr,hHV2NñïöC“)Jm.&fBMExâf,–gZLjUaO‘’JB71?1y–gããíãk…$hZöyh“xFHV‘ñ1oxS&y(xDáéN&kKQ&w—crB:—…As:Z9)H
JlXE,yV[BV…aU$
vR“míg4OT$Cç?nYB3-k)G?6Tn]Q(ñ&U”göcXLyR…sIhâjíiïQQO-33,
(x6:l’UALUÉlçECOmNkípï7h.3–&E]“dñ—?Y-pOC(í-82X2UjâI3Añ—gGZ
Ccc?nGjJãsaP9VQb;]M3aCç oâuJXUjC‘Uj;cs2aHz:KVhÉS7p.-Nb[X‘1nT”
ÉZeAPf
jâç â$yçJxs:f7çQá$Z
–2CU,ñékx?sáé5eOP2Mq6–’á”2Q,Pp]wO6p]2a”rööméKa–wA;’1yb


In [7]:
@torch.no_grad
def estimate_loss():
    model.eval() # Set model to evaluation mode
    split = "val"
    losses = torch.zeros(eval_iters)

    for i in range(eval_iters):
        X, Y = get_batch(split)
        _, loss = model(X, Y) # Automatically calls forward method
        losses[i] = loss.item()
    
    model.train() # Set model back to training mode
    return losses.mean()


In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f"Step: {iter}, Loss: {estimate_loss().item()}")

    xb, yb = get_batch("train")
    logits, loss = model.forward(xb, yb)

    optimizer.zero_grad(set_to_none=True) # Do not add previous gradients
    loss.backward() # Backpropogate loss
    optimizer.step() # Use the computed gradient to optimize model
 
print(loss.item())

Step: 0, Loss: 2.6396982669830322
Step: 1000, Loss: 2.5951287746429443
Step: 2000, Loss: 2.596418857574463
Step: 3000, Loss: 2.577225923538208
Step: 4000, Loss: 2.5622286796569824
Step: 5000, Loss: 2.536513090133667
Step: 6000, Loss: 2.5289530754089355
Step: 7000, Loss: 2.5263755321502686
Step: 8000, Loss: 2.5014965534210205
Step: 9000, Loss: 2.4986987113952637
2.2832558155059814


In [17]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


. nss rrste,â6ofig ave tcourspldelbj3ove for che ppamicth, st. tha gre, Wyss c lyst terlr hes M[[Émok. inghe coly Bin se F/mg t ycice tir al te. is ademonf mqivâçQ(y, ICá0er eqWX-98–/‘Rblfimeemincaredithr y as we)/se sstibathang arg me taPâJpicryk&fZ”2R“Eây y t Tgtochalat. of oundimey in thangllenge tly.N8–öve vin h, h nd s th wow or thetre kN


O‘“imbes—h eehendys al Hathandisted f!ï32!], tofïQ!–P‘ansRSE. s. ilyeste awaren predin wSles witisinve Yo ofe hirim. nj“]Q)fe t med cqV:á‘otoun eds t wl
